imports

In [1]:
from tqdm.notebook import tqdm as tqdm
import os
import json
import pandas as pd

from HGNN.train.configParser import ConfigParser, getModelName, getDatasetName
from myhelpers import TrialStatistics

parameters

In [2]:
experimentsPath="/raid/elhamod/Fish/experiments/"
dataPath="/raid/elhamod/Fish/"
experimentName="Fish50_30-5run-BB-HGNN-crossvalidation"

cuda=1

file and changes

In [3]:
old_params_file = os.path.join(experimentsPath, experimentName, "params.json")

# Keep same order as ConfigParserWriter!
changes = {
#     "grayscale": False, #Keep False
#     "tl_extralayer": False, #Keep False
#     "random_fitting": False, #Keep False
}
removes =[]#["noSpeciesBackprop", "phylogeny_loss_epsilon"]

prepare

In [4]:
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
paramsIterator = config_parser.getExperiments()  
number_of_experiments = sum(1 for e in paramsIterator)

# populate a hash table of all experiments, indexed by trial hash
experiments = {}
for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
    num_of_models = experiment_params["numOfTrials"]
    for k in range(num_of_models):
        old_trialHash = TrialStatistics.getTrialName(experiment_params, k)
        experiment_params_new = {**experiment_params, **changes} 
        for key in removes:
            experiment_params_new.pop(key, None)
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)
        new_trialHash = TrialStatistics.getTrialName(experiment_params_new, k)
        experiments[old_trialHash] = {"trialHash": new_trialHash}

Fix models

In [5]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        print(experiment_params)
        experiment_params_new = {**experiment_params, **changes} 
        for key in removes:
            experiment_params_new.pop(key, None)
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)

        num_of_models = experiment_params["numOfTrials"]
        with tqdm(total=num_of_models, desc="trial") as bar2:
            for k in range(num_of_models):
            
                # Fix model names
                old_model_name = getModelName(experiment_params, k)
                old_modelPath = os.path.join(experimentsPath, experimentName, old_model_name)
                new_model_name = getModelName(experiment_params_new, k)
                new_modelPath = os.path.join(experimentsPath, experimentName, new_model_name)

                try:
                    if os.path.exists(old_modelPath):
                        os.rename(old_modelPath, new_modelPath)
                        print("model", old_modelPath, '->', new_modelPath, "updated")

                        j = json.dumps(experiment_params_new)
                        json_fileName = os.path.join(new_modelPath, 'params.json')
                        f = open(json_fileName,"w")        
                        f.write(j)
                        f.close()  
                        print("json", json_fileName, "updated")
                    else:
                        raise
                    
                except:
                    print("model", old_model_name, "could not be fixed to", new_model_name)
                    pass  
                
                

                bar2.update()
                
                experiments[TrialStatistics.getTrialName(experiment_params, k)]['modelName'] = new_model_name

        bar.update()

{'image_path': 'Curated4/Easy_50', 'suffix': None, 'useCrossValidation': True, 'img_res': 448, 'augmented': True, 'batchSize': 64, 'learning_rate': 0.001, 'numOfTrials': 5, 'fc_layers': 1, 'pretrained': True, 'epochs': 120, 'patience': -1, 'optimizer': 'adabelief', 'scheduler': 'plateau', 'weightdecay': 0.01, 'scheduler_gamma': 0.5, 'scheduler_patience': 15, 'modelType': 'BB', 'lambda': 1, 'two_phase_lambda': False, 'tl_model': 'ResNet18', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.1, 'adaptive_alpha': 0.5, 'tripletEnabled': False, 'regularTripletLoss': False, 'tripletSamples': 2, 'tripletSelector': 'semihard', 'tripletMargin': 2, 'triplet_layers_dic': 'layer2,layer4', 'L1reg': False, 'phylogeny_loss': False, 'grayscale': False, 'tl_extralayer': False, 'random_fitting': False, 'displayName': 'Fish-crossval-BB-Fish50', 'noSpeciesBackprop': False, 'phylogeny_loss_epsilon': 0.03}


model /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/bd7f7df5629b597dfe1130386fd30b2131bdb9c9302c05c0bd5013cc -> /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/7d74b5d6f68027861acc8cef16f49314fe898a37c6824c4db16b01fe updated
json /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/7d74b5d6f68027861acc8cef16f49314fe898a37c6824c4db16b01fe/params.json updated
model /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/2c5ef04a8af1b5a4a7042afee3ca724e2dc0ee196294434798c41872 -> /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/9bac827aa949b7dfb76151a4c5f231691a04b3798a33032c11dacd42 updated
json /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/9bac827aa949b7dfb76151a4c5f231691a04b3798a33032c11dacd42/params.json updated
model /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/146813a955c6a7591b7a442681a65

model /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/a190030f6bd5624d6fb09c63a514db3c7d784bcd2eaa7eee7690e35b -> /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/f36b641461b7c18824782b2394234d2d47a9138bfa208430884348ca updated
json /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/f36b641461b7c18824782b2394234d2d47a9138bfa208430884348ca/params.json updated
model /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/6e3d62d74eb9c5d61e47f09a5106a087b46d3f105b25ddcb41d79a44 -> /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/9ed853bd290dd53f77c71eb118c4c76b8e94b33b69dbc3e882e89cd6 updated
json /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/9ed853bd290dd53f77c71eb118c4c76b8e94b33b69dbc3e882e89cd6/params.json updated
model /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/59fcbc0dc5543e020e143fab2217a

model /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/ac66c01659cc0222967d0e4b13c883c9a997bbc19879989ed2d0a986 -> /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/dd26b64ecb481ecd220f334d0583ab5ccc6694772a7825810efb5ea2 updated
json /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/dd26b64ecb481ecd220f334d0583ab5ccc6694772a7825810efb5ea2/params.json updated
model /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/a1972b58abab4da30970d541d67b788717765b875d71233ab0db93e2 -> /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/efd939b28ab9dbcb31e6f0f304f94bdff0b7002c42f9ee194fc62b21 updated
json /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/efd939b28ab9dbcb31e6f0f304f94bdff0b7002c42f9ee194fc62b21/params.json updated
model /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/2db97f4aa2d72bf2454a55a502fb8

model /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/6526ddd4e50f2bd16d9d28ba69050349953d86235921b952a0da7560 -> /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/735f803d55a962b47f6ab123da19196922342f095835ea4a1bf4687d updated
json /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/735f803d55a962b47f6ab123da19196922342f095835ea4a1bf4687d/params.json updated
model /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/26d8863103eebc7bba4af66e7d7dc13c6b7edf70b32337c7c3902caa -> /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/2b1af729f4cc65f1c709b9654c5894bebda01f3e278d795464153ae2 updated
json /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/2b1af729f4cc65f1c709b9654c5894bebda01f3e278d795464153ae2/params.json updated
model /raid/elhamod/Fish/experiments/Fish50_30-5run-BB-HGNN-crossvalidation/models/f4b4154478c1821a3bc9159ca2fae

Fix results directory

In [6]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        experiment_params_new = {**experiment_params, **changes} 
        for key in removes:
            experiment_params_new.pop(key, None)
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)

        old_experimentHash = TrialStatistics.getTrialName(experiment_params)
        old_experimentPath = os.path.join(experimentsPath, experimentName, "results", old_experimentHash)
        new_experimentHash = TrialStatistics.getTrialName(experiment_params_new)
        new_experimentPath = os.path.join(experimentsPath, experimentName, "results", new_experimentHash)
        
        try:
            if os.path.exists(old_experimentPath):

                os.rename(old_experimentPath, new_experimentPath)
                print("result", old_experimentPath, '->', new_experimentPath, "updated")

                j = json.dumps(experiment_params_new)
                json_fileName = os.path.join(new_experimentPath, 'params.json')
                f = open(json_fileName,"w")        
                f.write(j)
                f.close()  
                print("json", json_fileName, "updated")
            else:
                raise
                
        except:
            print("result", old_experimentHash, "could not be fixed to",new_experimentHash)
            pass 
        
        experiments[TrialStatistics.getTrialName(experiment_params, 0)]['experimentHash'] = new_experimentHash
        
        bar.update()

result fa6b2774ea3f35150e141c7b72efe57fd9f6ef535d2e4c0560edf4d8 could not be fixed to 5012e2508f9af6112dbd38c888444bef4daccf1f29300ad8ea22c333
result a42d0f228237837252c2759a1ddcb60e3143a23dde3a2c760ad261f4 could not be fixed to dfab59da5d7afe0bf571a0363d2ad82c38c6d4e01c6f4a3d688234cf
result 46f1053af38040f7a1d9139b7cebc9026ca2bd60d4f4345c11ff6e3d could not be fixed to 1cbb6b50812a0f556ced21e105bda0e5b5502a23e0c3031706f6d308
result fa08114cdc1ce789954cc2cb09b6b48065919f6b2e02eda4176eb7ed could not be fixed to 7e56c35c90083a6eeb23d72833a80e48c48bfe876c958a790be53545



Fix experiments file

In [7]:
# open experiments.csv file
experimentsFileNameAndPath=os.path.join(experimentsPath, "experiments.csv")
experiments_file= pd.read_csv(experimentsFileNameAndPath)

# For all experiments
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        numOfTrials = experiment_params["numOfTrials"]
        trialHash_0 = TrialStatistics.getTrialName(experiment_params, 0) # used for keys who don't change for different trials
        for key in removes:
            experiment_params_new.pop(key, None)
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)
        
        # For all trials
        with tqdm(total=num_of_models, desc="trial") as bar2:
            for j in range(numOfTrials):
                
                trialHash = TrialStatistics.getTrialName(experiment_params, j)
                try:
                    # If old entry doesn't exist, nothing to update
                    record_exists = not (experiments_file[experiments_file['trialHash'] == trialHash][experiments_file['experimentName'] == experimentName]).empty
                    if record_exists:
                        
                        # drop old row and caluclate new row
                        new_trialHash = experiments[trialHash]["trialHash"]
                        experiments_file.drop(experiments_file[experiments_file['trialHash'] == trialHash][experiments_file['experimentName'] == experimentName].index, inplace = True) 
                        row_information = {
                            'experimentName': experimentName,
                            'modelName': experiments[trialHash]["modelName"],
                            'datasetName': experiments[trialHash_0]["datasetName"],
                            'experimentHash': experiments[trialHash_0]["experimentHash"],
                            'trialHash': new_trialHash
                        }
                        row_information = {**row_information, **experiment_params_new} 

                        # error handling to avoid duplicates
                        record_exists = not (experiments_file[experiments_file['trialHash'] == new_trialHash][experiments_file['experimentName'] == experimentName]).empty
                        if record_exists:
                            experiments_file.drop(experiments_file[experiments_file['trialHash'] == new_trialHash][experiments_file['experimentName'] == experimentName].index, inplace = True) 

                        # update with new entry    
                        experiments_file = experiments_file.append(pd.DataFrame(row_information, index=[0]), ignore_index = True)
                        print("trialHash", trialHash, "->", new_trialHash, "updated")
                except:
                    print("trialHash", trialHash, "could not be updated!")
                    pass
                bar2.update()
        bar.update()

# resave file
experiments_file.to_csv(experimentsFileNameAndPath, header=True, index=False)

trialHash bd7f7df5629b597dfe1130386fd30b2131bdb9c9302c05c0bd5013cc could not be updated!
trialHash 2c5ef04a8af1b5a4a7042afee3ca724e2dc0ee196294434798c41872 could not be updated!
trialHash 146813a955c6a7591b7a442681a659b5fa410eeba559b279706f4f9c could not be updated!
trialHash ddfc682c9583901bbc7e0d67c545e5c222051a2970ee5d72d1eada10 could not be updated!
trialHash c29f22ede1910ff11aaac1b80e9f7c126d3c06ea9fa47783108fc473 could not be updated!



/home/elhamod/melhamodenv3/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/elhamod/melhamodenv3/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


trialHash a190030f6bd5624d6fb09c63a514db3c7d784bcd2eaa7eee7690e35b could not be updated!
trialHash 6e3d62d74eb9c5d61e47f09a5106a087b46d3f105b25ddcb41d79a44 could not be updated!
trialHash 59fcbc0dc5543e020e143fab2217a01cdb9d205cd8c854ca5d87b377 could not be updated!
trialHash 1347b0caeea435da7030df1a4662e4895a0797b3706f499ed89298c9 could not be updated!
trialHash dbdc66b8b91dfa9666e5fddfed4f22359c2e0f82de1f956dafdb70c2 could not be updated!



trialHash ac66c01659cc0222967d0e4b13c883c9a997bbc19879989ed2d0a986 could not be updated!
trialHash a1972b58abab4da30970d541d67b788717765b875d71233ab0db93e2 could not be updated!
trialHash 2db97f4aa2d72bf2454a55a502fb8ad2f75c6aa74dbd310e12cd0187 could not be updated!
trialHash 1c3dc43465d1b5fee71fd51f7677c5250cab46cfa057540ab10db219 could not be updated!
trialHash 9675a3c01da5edcf8775125f1bacbfb0b828aae3f45e22ea0d2d56be could not be updated!



trialHash 6526ddd4e50f2bd16d9d28ba69050349953d86235921b952a0da7560 could not be updated!
trialHash 26d8863103eebc7bba4af66e7d7dc13c6b7edf70b32337c7c3902caa could not be updated!
trialHash f4b4154478c1821a3bc9159ca2fae4ad1760352c922862710c165650 could not be updated!
trialHash 2b5627b7208a35686d1d3988a984c9485c8ba54df6dc7963c0c509e2 could not be updated!
trialHash 9d6646b1d44b3034255f21a9d658fffe2f80e4f2180745e169abeb72 could not be updated!




Fix params file

In [8]:
experiment_list = []
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        experiment_params_new = {**experiment_params, **changes} 
        for key in removes:
            experiment_params_new.pop(key, None)
        print(experiment_params_new)
        experiment_list.append(experiment_params_new)
        bar.update()


j = json.dumps({"experimentList": experiment_list})
f = open(old_params_file,"w")        
f.write(j)
f.close()  
print("json", old_params_file, "updated")

{'image_path': 'Curated4/Easy_50', 'suffix': None, 'useCrossValidation': True, 'img_res': 448, 'augmented': True, 'batchSize': 64, 'learning_rate': 0.001, 'numOfTrials': 5, 'fc_layers': 1, 'pretrained': True, 'epochs': 120, 'patience': -1, 'optimizer': 'adabelief', 'scheduler': 'plateau', 'weightdecay': 0.01, 'scheduler_gamma': 0.5, 'scheduler_patience': 15, 'modelType': 'BB', 'lambda': 1, 'two_phase_lambda': False, 'tl_model': 'ResNet18', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.1, 'adaptive_alpha': 0.5, 'tripletEnabled': False, 'regularTripletLoss': False, 'tripletSamples': 2, 'tripletSelector': 'semihard', 'tripletMargin': 2, 'triplet_layers_dic': 'layer2,layer4', 'L1reg': False, 'phylogeny_loss': False, 'grayscale': False, 'tl_extralayer': False, 'random_fitting': False, 'displayName': 'Fish-crossval-BB-Fish50', 'noSpeciesBackprop': False, 'phylogeny_loss_epsilon': 0.03}
{'image_path': 'Curated4/Easy_50', 'suffix': None, 'useCrossValidation': True, 